In [4]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [5]:
def parse_yolo_annotation(annotation_file, image_path):
    annotations = []
    with open(annotation_file, 'r') as f:
        for line in f:
            data = line.strip().split()
            class_id = {0:'neuron'}
            image = cv2.imread(image_path)
            image_height, image_width, _ = image.shape
            x_center, y_center, width, height =map(float, data[1:])
            # Перевод относительных координат в абсолютные пиксельные координаты
            x_center_px = int(x_center * image_width)
            y_center_px = int(y_center * image_height)
            width_px = int(width * image_width)
            height_px = int(height * image_height)
    
    # Вычисление координат bbox
            x1 = int(x_center_px - width_px / 2)
            y1 = int(y_center_px - height_px / 2)
            x2 = int(x_center_px + width_px / 2)
            y2 = int(y_center_px + height_px / 2)
            annotations.append((x1, y1, x2, y2))
            
    return annotations

In [6]:
def crop_annotated_regions(image_path, annotation_file, output_folder):

    annotations = parse_yolo_annotation(annotation_file, image_path)
    image = cv2.imread(image_path)

    filename = os.path.basename(image_path)[:-4] # Убрал тип jpg

    x1_first, y1_first, x2_first, y2_first = annotations[0]
    x1_last, y1_last, x2_last, y2_last = annotations[-1]

     # Определение координат промежутка между первой и последней аннотациями
    x1_interval = max(x1_first, x1_last)
    y1_interval = max(y1_first, y1_last)
    x2_interval = min(x2_first, x2_last)
    y2_interval = min(y2_first, y2_last)

    cropped_interval = image[y1_interval:y2_interval, x1_interval:x2_interval]
    output_filename = os.path.join(output_folder, f"cropped_{filename}.jpg")

    cv2.imwrite(output_filename, cropped_interval)
    
    print(f"Сохранено обрезанное изображение: {output_filename}")

In [15]:

# Пример использования прямая линия
image_path = "C:\\Users\\minik\\Downloads\\Neuron.v4i.yolov8_640_Ruslan\\train\images\\snaps-52-_jpg.rf.8167b8d8003962ea80f964d3adad4f91.jpg"
annotation_file = 'C:\\Users\\minik\\Downloads\\Neuron.v4i.yolov8_640_Ruslan\\train\\labels\\Snap-11082_jpg.rf.d5d93d0ef16da65767aefeaa288e4fc7.txt'
output_folder = 'C:\\Users\\minik\\Downloads\\Neuron.v4i.yolov8_640_Ruslan\\cut_images'
crop_annotated_regions(image_path, annotation_file, output_folder)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:786: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [14]:

# Пример использования косая линия
image_path = 'C:\\Users\\minik\\Downloads\\Neuron.v4i.yolov8_640_Ruslan\\train\\images\\snaps-52-_jpg.rf.8167b8d8003962ea80f964d3adad4f91.jpg'
annotation_file = 'C:\\Users\\minik\\Downloads\\Neuron.v4i.yolov8_640_Ruslan\\train\\labels\\snaps-52-_jpg.rf.8167b8d8003962ea80f964d3adad4f91.txt'
output_folder = 'C:\\Users\\minik\\Downloads\\Neuron.v4i.yolov8_640_Ruslan\\cut_images'
crop_annotated_regions(image_path, annotation_file, output_folder)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:786: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


### Попробуем с маской

In [11]:
def parse_yolo_annotation(annotation_file, image_path):
    annotations = []
    with open(annotation_file, 'r') as f:
        for line in f:
            data = line.strip().split()
            class_id = {0:'neuron'}
            image = cv2.imread(image_path)
            image_height, image_width, _ = image.shape
            x_center, y_center, width, height =map(float, data[1:])
            # Перевод относительных координат в абсолютные пиксельные координаты
            x_center_px = int(x_center * image_width)
            y_center_px = int(y_center * image_height)
            width_px = int(width * image_width)
            height_px = int(height * image_height)
    
    # Вычисление координат bbox
            x1 = int(x_center_px - width_px / 2)
            y1 = int(y_center_px - height_px / 2)
            x2 = int(x_center_px + width_px / 2)
            y2 = int(y_center_px + height_px / 2)
            annotations.append((x1, y1, x2, y2))
    return annotations

annotations = parse_yolo_annotation(annotation_file, image_path)

In [12]:
for an in annotations:
    print (an[])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3883951266.py, line 2)

In [13]:
# Извлечение координат центров прямоугольников
x_values = [(rect[0] + rect[2])  for rect in annotations]
y_values = [(rect[1] + rect[3])  for rect in annotations]

# Выполнение линейной регрессии
slope, intercept = np.polyfit(x_values, y_values, 1)

# Построение линии тренда
x_trend = np.linspace(min(x_values), min(x_values), 100)
y_trend = slope * x_trend + intercept

# Построение графика
plt.imshow((image))
plt.figure(figsize=(8, 6))
plt.scatter(x_values, y_values, label='Прямоугольники')
plt.plot(x_trend, y_trend, color='red', linestyle='--', label='Линия тренда')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Линия тренда из прямоугольников')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'image' is not defined

In [ ]:
def interpolate_point(point1, point2, t):
    x = int(point1[0] * (1 - t) + point2[0] * t)
    y = int(point1[1] * (1 - t) + point2[1] * t)
    return x, y

def create_line_coordinates(image_shape, annotations, step=10):
    line_coordinates = []

    # Определяем точки, через которые проходит линия
    points = []
    for annotation in annotations:
        x1, y1, x2, y2 = annotation
        points.append(((x1 + x2) // 2, (y1 + y2) // 2))

    # Находим координаты точек на линии с заданным шагом
    for i in range(len(points) - 1):
        point1 = points[i]
        point2 = points[i + 1]
        length = np.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)
        num_steps = int(length / step)
        for j in range(num_steps):
            t = j / num_steps
            interpolated_point = interpolate_point(point1, point2, t)
            line_coordinates.append(interpolated_point)
        line_coordinates.append(point2)

    return line_coordinates

def crop_image_along_line(image_path, annotations, step=10):
    # Загрузка изображения
    image = cv2.imread(image_path)

    # Создаем координаты линии
    line_coordinates = create_line_coordinates(image.shape, annotations, step)

    # Находим минимальные и максимальные координаты для обрезки изображения
    min_x = min(coord[0] for coord in line_coordinates)
    max_x = max(coord[0] for coord in line_coordinates)
    min_y = min(coord[1] for coord in line_coordinates)
    max_y = max(coord[1] for coord in line_coordinates)

    # Обрезаем изображение
    cropped_image = image[min_y:max_y, min_x:max_x]

    return cropped_image

In [ ]:
# Обрезка изображения вдоль линии с шагом в 300 пикселей, проходящей через все аннотации
cropped_image = crop_image_along_line(image_path, annotations, step=10)

# Сохранение обрезанного изображения
cv2.imwrite('cropped_image_along_line_with_step.jpg', cropped_image)

True